In [1]:
import numpy as np
import pandas as pd
import scipy.constants as consts
import matplotlib.pyplot as plt
from io import StringIO
import glob
import os, sys
import re
plt.rcParams['figure.dpi']= 300

In [2]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 5),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)
# set wanted parameters for plots

In [ ]:
def name_dat_file():
    filenames = [f for f in glob.glob(directory + '*.dat')]
    return filenames

Sort filenames nicely

In [3]:
def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]"""
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    #Sort the given list in the way that humans expect
    l.sort(key=alphanum_key)
    return l

### Read files into dataframe

In [5]:
def energ_to_df(filename):
    column_names = ['energy','variance', 't_cpu', 'acceptance']
    df = pd.read_csv(filename,names=column_names, delim_whitespace=True,comment='#')
    return df

### Blocking

In [6]:
def block_mean(vec):
    return sum(vec)/len(vec)

def meanAndVariance(vec):
    mean = np.mean(vec)
    var = sum([i ** 2 for i in vec])/len(vec) - mean*mean
    return mean, var

In [7]:
def everything_is_awesome(data): # does the blocking on the dataframe
    n_blocks = 200
    block_size_min = 100
    block_size_max = len(data)/100
    block_step = int ((block_size_max - block_size_min + 1) / n_blocks)
    mean_vec = []
    var_vec = []
    block_sizes = []
    for i in range(0, n_blocks):
        mean_temp_vec = []
        start_point = 0
        end_point = block_size_min + block_step*i
        block_size = end_point
        block_sizes.append(block_size)

    mean_temp_vec.append(block_mean(data[start_point:end_point]))
    start_point = end_point
    end_point += block_size_min + block_step*i
    mean, var = meanAndVariance(mean_temp_vec)
    mean_vec.append(mean)
    var_vec.append(np.sqrt(var/(len(data)/float(block_size) - 1.0)))
    mean, var = meanAndVariance(data)

    return mean,var

#### Add $\sigma_b$ to a dataframe

In [9]:
def add_block(dfmain,dfen):
    meanlist = []
    stdlist = []
    tempmean, tempvar = everything_is_awesome(dfen['energy'])
    meanlist.append(tempmean)
    stdlist.append(np.sqrt(np.abs((tempvar))))
    dfmain['std_b'] = stdlist
    return dfmain

In [11]:
def plotenergy(df,legend_names):
    plt.plot(df['energy'])
    plt.xlabel('time')
    plt.ylabel('energy')
    plt.legend(legend_names)
    plt.show()